# Week 2 Project: Refining the Art of Sentiment Analysis at ModaMetric

Welcome to Week 2! The ModaMetric team is still buzzing from the achievements of last week. You've shown them the power of Metaflow and the potential of machine learning. However, there's more to explore, more to refine.

Once again, we’ll delve into the [Women's Ecommerce Clothing Reviews Dataset from Kaggle](https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews), the dataset that helped us unlock valuable insights for ModaMetric. Your mission is to further refine the sentiment analysis process, enabling ModaMetric to better understand the sentiments embedded in the customer reviews.

## Task 1: Orchestrating the Dance of Sentiment Analysis Models with Metaflow

In this task, you'll utilize Metaflow to train two sentiment analysis models: the baseline "majority class" classifier and your own custom model. The models will be trained simultaneously, flexing the power of Metaflow. Your task also involves tweaking the models' hyperparameters for optimal performance. Finally, you'll analyze the performance of these models using Metaflow's Client API. Here's how you'll proceed:

### Step 1: Constructing the Sentiment Analysis Workflows
Your first task is to construct the Metaflow workflows. Begin with the baseline "majority class" classifier and then move on to your custom model. Make sure your custom model includes steps for data preprocessing, model training, and evaluation. Feel free to use techniques from Week 1 and any other [resources](https://outerbounds.com/docs/nlp-tutorial-L2/) you find useful.

### Step 2: Parallel Training of Models
Having built the models, you'll use Metaflow to train them simultaneously. The race is on - can the custom model outshine the baseline? If you find yourself in a bind, you might find the [FlowSpec branching documentation](https://docs.metaflow.org/metaflow/basics#branch) useful.

### Step 3: The Hyperparameters Experiment
Once you've trained the models, it's time for some fine-tuning. Experiment with different hyperparameters such as learning rate, batch size, and number of epochs. Record the performance of each model under different hyperparameter combinations as Data Artifacts in Metaflow.

### Step 4: Results Analysis
With the experiments complete, it's time to analyze the results. Use Metaflow's Client API to fetch the data and create visualizations to compare the models' performances. The goal is to identify the best hyperparameters for each model.

By completing this task, you're not only refining the sentiment analysis process at ModaMetric but also honing your own skills in orchestrating complex machine learning workflows using Metaflow.


In [1]:
from collections import Counter
import pandas as pd
import numpy as np
from termcolor import colored
import matplotlib.pyplot as plt
import seaborn as sns
import string
import io
from sklearn.model_selection import train_test_split

# You can style your plots here, but it is not part of the project.
YELLOW = "#FFBC00"
GREEN = "#37795D"
PURPLE = "#5460C0"
BACKGROUND = "#F4EBE6"
colors = [GREEN, PURPLE]
custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.facecolor": BACKGROUND,
    "figure.facecolor": BACKGROUND,
    "figure.figsize": (8, 8),
}
sns_palette = sns.color_palette(colors, len(colors))
sns.set_theme(style="ticks", rc=custom_params)

In [2]:
# TODO: load the data.
df = pd.read_csv("./../data/Womens Clothing E-Commerce Reviews.csv", index_col=0)

# transformations
df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
df = df[~df.review_text.isna()]
df["review"] = df["review_text"].astype("str")
_has_review_df = df[df["review_text"] != "nan"]
reviews = _has_review_df["review_text"]

def labeling_function(row):
    if row['rating']>=4:
        return 1
    else:
        return 0

labels = _has_review_df.apply(labeling_function, axis=1)
df = pd.DataFrame({"label": labels, **_has_review_df})

# split into training and validation.
_df = pd.DataFrame({"review": reviews, "label": labels})
traindf, valdf = train_test_split(_df, test_size=0.2)

In [3]:
# TODO: build the majority class baseline model.
_df['label'].value_counts()
# TODO: find the majority class in the labels. 🤔
traindf['base_model'] = 1
valdf['base_model'] = 1
# TODO: score the model on valdf with a 2D metric space: sklearn.metrics.accuracy_score, sklearn.metrics.roc_auc_score
# Documentation on suggested model-scoring approach: https://scikit-learn.org/stable/modules/model_evaluation.html
from sklearn.metrics import accuracy_score, roc_auc_score

### Fit and score a baseline model on the data, log the acc and rocauc as artifacts.
base_acc = accuracy_score(valdf["label"], valdf["base_model"])
base_rocauc = roc_auc_score(valdf["label"], valdf["base_model"])

print(base_acc)
print(base_rocauc)

0.7716935305807021
0.5


In [89]:
%%writefile model.py
# TODO: modify this custom model to your liking. Check out this tutorial for more on this class: https://outerbounds.com/docs/nlp-tutorial-L2/
# TODO: train the model on traindf.
# TODO: score the model on valdf with _the same_ 2D metric space you used in previous cell.
# TODO: test your model works by importing the model module in notebook cells, and trying to fit traindf and score predictions on the valdf data!

import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

class NbowModel():
    def __init__(self, vocab_sz):
        self.vocab_sz = vocab_sz
        # Instantiate the CountVectorizer
        self.cv = CountVectorizer(
            min_df=.005, max_df = .75, stop_words='english', 
            strip_accents='ascii', max_features=self.vocab_sz
        )

        # Define the keras model
        inputs = tf.keras.Input(shape=(self.vocab_sz,), 
                                name='input')
        x = layers.Dropout(0.10)(inputs)
        x = layers.Dense(
            15, activation="relu",
            kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
        )(x)
        predictions = layers.Dense(1, activation="sigmoid",)(x)
        self.model = tf.keras.Model(inputs, predictions)
        opt = optimizers.Adam(lr=0.55e-3)

        self.model.compile(loss="binary_crossentropy", 
                           optimizer=opt, metrics=["accuracy"])

    def fit(self, X, y):
        res = self.cv.fit_transform(X).toarray()
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', patience=5,
                                                  restore_best_weights=True)

        self.model.fit(x=res, y=y, batch_size=32, 
                       epochs=100, callbacks=[early_stopping], validation_split=.2)
    
    def predict(self, X):
        res = self.cv.transform(X).toarray()
        return self.model.predict(res)
    
    def eval_acc(self, X, labels, threshold=.5):
        return accuracy_score(labels, 
                              self.predict(X) > threshold)
    
    def eval_rocauc(self, X, labels):
        return roc_auc_score(labels,  self.predict(X))

    @property
    def model_dict(self): 
        return {'vectorizer':self.cv, 'model': self.model}

    @classmethod
    def from_dict(cls, model_dict):
        "Get Model from dictionary"
        nbow_model = cls(len(
            model_dict['vectorizer'].vocabulary_
        ))
        nbow_model.model = model_dict['model']
        nbow_model.cv = model_dict['vectorizer']
        return nbow_model

Overwriting model.py


In [5]:
from model import NbowModel
import pandas as pd
model = NbowModel(200)
model.fit(X=traindf['review'], y=traindf['label'])

2023-10-27 17:48:28.560378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 17:48:30.557233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/workspace/mambaforge/envs/full-stack-metaflow-corise/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
453/453 [==============================] - 2s 3ms/step - loss: 0.5094 - accuracy: 0.7689 - val_loss: 0.4251 - val_accuracy: 0.7985
Epoch 2/100
453/453 [==============================] - 1s 2ms/step - loss: 0.4010 - accuracy: 0.8225 - val_loss: 0.3778 - val_accuracy: 0.8327
Epoch 3/100
453/453 [==============================] - 1s 2ms/step - loss: 0.3789 - accuracy: 0.8350 - val_loss: 0.3691 - val_accuracy: 0.8402
Epoch 4/100
453/453 [==============================] - 1s 2ms/step - loss: 0.3737 - accuracy: 0.8374 - val_loss: 0.3674 - val_accuracy: 0.8394
Epoch 5/100
453/453 [==============================] - 1s 2ms/step - loss: 0.3697 - accuracy: 0.8379 - val_loss: 0.3644 - val_accuracy: 0.8429
Epoch 6/100
453/453 [==============================] - 1s 2ms/step - loss: 0.3696 - accuracy: 0.8391 - val_loss: 0.3652 - val_accuracy: 0.8410
Epoch 7/100
453/453 [==============================] - 1s 2ms/step - loss: 0.3668 - accuracy: 0.8380 - val_loss: 0.3638 - val_accuracy: 0.8416

In [48]:
model_acc = model.eval_acc(X=valdf['review'].values, labels=valdf['label'].values)
model_rocauc = model.eval_rocauc(X=valdf['review'].values, labels=valdf['label'].values)

msg = 'Baseline Accuracy: {}\nBaseline AUC: {}'
print(msg.format(round(model_acc, 3), round(model_rocauc, 3)))

(4529,)
I bought this sweater the minute i saw it, and waited for over six weeks for it to come off of back order (it was further delayed from even what i was told when i first ordered it).
i was beyond excited when it finally arrived and wore the sweater the day after i received it... i had a long flight and thought it was the perfect option for keeping comfy on a plane.
i was so disappointed that by the time i got off my flight four hours later, the sweater had not one, not two, but three differen
142/142 [==============================] - 0s 940us/step
(4529,)
I bought this sweater the minute i saw it, and waited for over six weeks for it to come off of back order (it was further delayed from even what i was told when i first ordered it).
i was beyond excited when it finally arrived and wore the sweater the day after i received it... i had a long flight and thought it was the perfect option for keeping comfy on a plane.
i was so disappointed that by the time i got off my flight four

In [91]:
%%writefile baseline_challenge.py
# TODO: In this cell, write your BaselineChallenge flow in the baseline_challenge.py file.

from metaflow import (
    FlowSpec,
    step,
    Flow,
    current,
    Parameter,
    IncludeFile,
    card,
    current,
)
from metaflow.cards import Table, Markdown, Artifact, Image
import numpy as np
from dataclasses import dataclass

# TODO: Define your labeling function here.
labeling_function = lambda row: 1 if row['rating'] >= 4 else 0


@dataclass
class ModelResult:
    "A custom struct for storing model evaluation results."
    name: None
    params: None
    pathspec: None
    acc: None
    rocauc: None


class BaselineChallenge(FlowSpec):
    split_size = Parameter("split-sz", default=0.2)
    data = IncludeFile("data", default="../data/Womens Clothing E-Commerce Reviews.csv")
    kfold = Parameter("k", default=5)
    scoring = Parameter("scoring", default="accuracy")

    @step
    def start(self):
        import pandas as pd
        import io
        from sklearn.model_selection import train_test_split

        # load dataset packaged with the flow.
        # this technique is convenient when working with small datasets that need to move to remove tasks.
  
        df = pd.read_csv(io.StringIO(self.data))
        # Look up a few lines to the IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv').
        # You can find documentation on IncludeFile here: https://docs.metaflow.org/scaling/data#data-in-local-files

        # filter down to reviews and labels
        df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
        df = df[~df.review_text.isna()]
        df["review"] = df["review_text"].astype("str")
        _has_review_df = df[df["review_text"] != "nan"]
        reviews = _has_review_df["review_text"]
        labels = _has_review_df.apply(labeling_function, axis=1)
        self.df = pd.DataFrame({"label": labels, **_has_review_df})

        # split the data 80/20, or by using the flow's split-sz CLI argument
        _df = pd.DataFrame({"review": reviews, "label": labels})
        self.traindf, self.valdf = train_test_split(_df, test_size=self.split_size)
        print(f"num of rows in train set: {self.traindf.shape[0]}")
        print(f"num of rows in validation set: {self.valdf.shape[0]}")

        self.next(self.baseline, self.model)

    @step
    def baseline(self):
        "Compute the baseline"

        from sklearn.metrics import accuracy_score, roc_auc_score

        self._name = "baseline"
        params = "Always predict 1"
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        # TODO: predict the majority class
        self.traindf["base_model"] = 1
        self.valdf["base_model"] = 1
        # TODO: return the accuracy_score of these predictions
        acc = accuracy_score(self.valdf["label"], self.valdf["base_model"])

        # TODO: return the roc_auc_score of these predictions
        rocauc =  roc_auc_score(self.valdf["label"], self.valdf["base_model"])
        self.result = ModelResult("Baseline", params, pathspec, acc, rocauc)
        self.next(self.aggregate)

    @step
    def model(self):
        # TODO: import your model if it is defined in another file.
        from model import NbowModel
        self._name = "model"
        # NOTE: If you followed the link above to find a custom model implementation,
        # you will have noticed your model's vocab_sz hyperparameter.
        # Too big of vocab_sz causes an error. Can you explain why?
        self.hyperparam_set = [{"vocab_sz": 100}, {"vocab_sz": 300}, {"vocab_sz": 500}]
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        self.results = []
        for params in self.hyperparam_set:
            model = NbowModel(params["vocab_sz"])
            model.fit(X=self.traindf["review"], y=self.traindf["label"])
            # TODO: evaluate your custom model in an equivalent way to accuracy_score.
            acc = model.eval_acc(X=self.valdf['review'].values, labels=self.valdf['label'].values)
            # TODO: evaluate your custom model in an equivalent way to roc_auc_score.
            rocauc = model.eval_rocauc(X=self.valdf['review'].values, labels=self.valdf['label'].values)
            self.results.append(
                ModelResult(
                    f"NbowModel - vocab_sz: {params['vocab_sz']}",
                    params,
                    pathspec,
                    acc,
                    rocauc,
                )
            )

        self.next(self.aggregate)

    @step
    def aggregate(self, inputs):
        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == "__main__":
    BaselineChallenge()

Overwriting baseline_challenge.py


In [92]:

! python baseline_challenge.py run

14032.16s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Metaflow 2.9.7.2+ob(v1) executing BaselineChallenge for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Including file ../data/Womens Clothing E-Commerce Reviews.csv of size 8MB 
2023-10-27 21:41:54.679 Workflow starting (run-id 30), see it in the UI at https://ui-pw-1910845203.outerbounds.dev/BaselineChallenge/30
2023-10-27 21:41:54.861 [30/start/135 (pid 4946)] Task is starting.
2023-10-27 21:41:58.999 [30/start/135 (pid 4946)] num of rows in train set: 18112
2023-10-27 21:42:02.627 [30/start/135 (pid 4946)] num of rows in validation set: 4529
2023-10-27 21:42:03.031 [30/start/135 (pid 4946)] Task finished successfully.
2023-10-27 21:42:03.488 [30/baseline/136 (pid 5048)] Task is starting.
2023-10-27 21:42:03.654 [30/model/137 (pid 5051)] Task is starting.
2023-10-27 21:42:05.815 [30/model/137 (pid 5051)] 2023-10-27 21:42:05.815741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAP

## Task 2: Mastering the Art of Anticipation: Failures and Remedies in ModaMetric's Machine Learning Journey

In this task, your challenge is to step into the role of a foresightful data scientist at ModaMetric, where you'll be anticipating potential pitfalls in the sentiment analysis classifier project. Not just that, but you'll also be charting out strategies to steer clear of these hitches. Here's how you'll navigate through:

### Step 1: Forecasting Potential Failure Modes

The key to overcoming challenges is to anticipate them. Start by picturing possible failure scenarios from an engineering perspective. For instance, you might think about problems like overfitting to the training data or biases in the data. Remember, the first step to finding a solution is acknowledging the problem.

    - There are a wide range of words in our model. This can lead to a large number of parameters that the model needs to learn, making it more susceptible to overfitting.
    - Use techniques like k-fold cross-validation to assess the model's performance across multiple subsets of the data. Overfitting is evident when the model performs well on some folds but poorly on others.

### Step 2: Strategizing to Mitigate Failure Modes

Having identified the potential obstacles, your next task is to devise counter-strategies. Consider what steps you'd take to address the problem if it arises. For instance, to counter overfitting, you could employ regularization techniques such as L1 or L2 regularization. Think of this step as drawing up a contingency plan.

    - Regularization: Apply regularization techniques such as L1 or L2 regularization to penalize overly complex model parameters, so the model can focus on the most important features and prevents it from fitting the noise in the data.
    - Early Stopping: Monitor the model's performance on a validation set during training and stop training when the performance begins to degrade, indicating overfitting.


### Step 3: Planning Ahead to Dodge Failure Modes

Beyond reactive strategies, you also need a proactive plan. What could you have done at the outset to avoid these potential pitfalls? Could you have collected a more diverse dataset to reduce bias? Or experimented with different model architectures? The goal is to minimize reactive measures and maximize foresight.


The proactive plan is as follow:
    - Collect a Diverse Dataset: To reduce bias and ensure that your model generalizes well, start by collecting a diverse and representative dataset. A diverse dataset should encompass a wide range of examples and real-world variations, including different demographic groups, contexts, and scenarios.

    - Data Preprocessing: Carefully preprocess the text data to handle common issues like misspellings, punctuation, and tokenization. make sure the data is clean and consistent.

    - Evaluate Different Architectures: Experiment with different model architectures and approaches, including simple and complex models. By exploring various options, you can gain insights into what works best for your specific problem.


This task emphasizes the importance of anticipation in machine learning projects. By identifying possible failure modes and crafting mitigation strategies, you'll be preparing yourself for a smooth-sailing machine learning journey at ModaMetric.


## Task 3: Bringing ML Results to Life: ModaMetric's Visualization Adventure with MF Cards

It's time for you to go beyond the code and transform data into a visual narrative. As a member of ModaMetric's data science team, your next mission is to enhance the existing flow in your `baseline_challenge.py` file. Add a new layer that gathers the results from all the hyperparameter tuning jobs. But that's not all - you're also going to breathe life into this aggregated data by creating a data visualization using Metaflow cards. Here's what you need to do:

### Step 1: Extend Your Flow

Your first challenge is to add another level to your existing `baseline_challenge.py` file. This new addition should be able to collate all the outcomes from your various hyperparameter tuning jobs. 

### Step 2: Log Results and Create Data Visualization

Once you've collected the outcomes, it's time to log the results in a structured way. Then, you're going to take this information and create a compelling data visualization using Metaflow cards. Remember, a picture is worth a thousand numbers. With these visual insights, you'll be enabling ModaMetric to understand the performance of your machine learning model in a glance.

This task is your opportunity to blend your technical skills with creative thinking. By visualizing your ML results, you're not only making the data more digestible but also contributing to ModaMetric's data-driven decision-making process.

In [95]:
%%writefile baseline_challenge.py
# TODO: In this cell, write your BaselineChallenge flow in the baseline_challenge.py file.

from metaflow import (
    FlowSpec,
    step,
    Flow,
    current,
    Parameter,
    IncludeFile,
    card,
    current,
)
from metaflow.cards import Table, Markdown, Artifact, Image
import numpy as np
from dataclasses import dataclass

# TODO: Define your labeling function here.
labeling_function = lambda row: 1 if row['rating'] >= 4 else 0


@dataclass
class ModelResult:
    "A custom struct for storing model evaluation results."
    name: None
    params: None
    pathspec: None
    acc: None
    rocauc: None


class BaselineChallenge(FlowSpec):
    split_size = Parameter("split-sz", default=0.2)
    data = IncludeFile("data", default="Womens Clothing E-Commerce Reviews.csv")
    kfold = Parameter("k", default=5)
    scoring = Parameter("scoring", default="accuracy")

    @step
    def start(self):
        import pandas as pd
        import io
        from sklearn.model_selection import train_test_split

        # load dataset packaged with the flow.
        # this technique is convenient when working with small datasets that need to move to remove tasks.
        # TODO: load the data.
        df = pd.read_csv(io.StringIO(self.data))
        # Look up a few lines to the IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv').
        # You can find documentation on IncludeFile here: https://docs.metaflow.org/scaling/data#data-in-local-files

        # filter down to reviews and labels
        df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
        df = df[~df.review_text.isna()]
        df["review"] = df["review_text"].astype("str")
        _has_review_df = df[df["review_text"] != "nan"]
        reviews = _has_review_df["review_text"]
        labels = _has_review_df.apply(labeling_function, axis=1)
        self.df = pd.DataFrame({"label": labels, **_has_review_df})

        # split the data 80/20, or by using the flow's split-sz CLI argument
        _df = pd.DataFrame({"review": reviews, "label": labels})
        self.traindf, self.valdf = train_test_split(_df, test_size=self.split_size)
        print(f"num of rows in train set: {self.traindf.shape[0]}")
        print(f"num of rows in validation set: {self.valdf.shape[0]}")

        self.next(self.baseline, self.model)

    @step
    def baseline(self):
        "Compute the baseline"

        from sklearn.metrics import accuracy_score, roc_auc_score

        self._name = "baseline"
        params = "Always predict 1"
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        # TODO: predict the majority class
        self.traindf["base_model"] = 1
        self.valdf["base_model"] = 1
        # TODO: return the accuracy_score of these predictions
        acc = accuracy_score(self.valdf["label"], self.valdf["base_model"])
        # TODO: return the roc_auc_score of these predictions
        rocauc =  roc_auc_score(self.valdf["label"], self.valdf["base_model"])

        self.result = ModelResult("Baseline", params, pathspec, acc, rocauc)
        self.next(self.aggregate)

    @step
    def model(self):
        # TODO: import your model if it is defined in another file.

        from model import NbowModel
        self._name = "model"
        # NOTE: If you followed the link above to find a custom model implementation,
        # you will have noticed your model's vocab_sz hyperparameter.
        # Too big of vocab_sz causes an error. Can you explain why?
        self.hyperparam_set = [{"vocab_sz": 50}, {"vocab_sz": 100}, {"vocab_sz": 300}, {"vocab_sz": 500}]
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        self.results = []
        for params in self.hyperparam_set:
            model = NbowModel(params["vocab_sz"])
            model.fit(X=self.traindf["review"], y=self.traindf["label"])
            # TODO: evaluate your custom model in an equivalent way to accuracy_score.
            acc = model.eval_acc(X=self.valdf['review'].values, labels=self.valdf['label'].values)
            # TODO: evaluate your custom model in an equivalent way to roc_auc_score.
            rocauc = model.eval_rocauc(X=self.valdf['review'].values, labels=self.valdf['label'].values)
            self.results.append(
                ModelResult(
                    f"NbowModel - vocab_sz: {params['vocab_sz']}",
                    params,
                    pathspec,
                    acc,
                    rocauc,
                )
            )

        self.next(self.aggregate)

    def add_one(self, rows, result, df):
        "A helper function to load results."
        rows.append(
            [
                Markdown(result.name),
                Artifact(result.params),
                Artifact(result.pathspec),
                Artifact(result.acc),
                Artifact(result.rocauc),
            ]
        )
        df["name"].append(result.name)
        df["accuracy"].append(result.acc)
        return rows, df

    @card(
        type="corise"
    )   # TODO: Set your card type to "corise".
    # I wonder what other card types there are?
    # https://docs.metaflow.org/metaflow/visualizing-results
    # https://github.com/outerbounds/metaflow-card-altair/blob/main/altairflow.py
    @step
    def aggregate(self, inputs):
        import seaborn as sns
        import matplotlib.pyplot as plt
        from matplotlib import rcParams

        rcParams.update({"figure.autolayout": True})

        rows = []
        violin_plot_df = {"name": [], "accuracy": []}
        for task in inputs:
            if task._name == "model":
                for result in task.results:
                    print(result)
                    rows, violin_plot_df = self.add_one(rows, result, violin_plot_df)
            elif task._name == "baseline":
                print(task.result)
                rows, violin_plot_df = self.add_one(rows, task.result, violin_plot_df)
            else:
                raise ValueError("Unknown task._name type. Cannot parse results.")

        current.card.append(Markdown("# All models from this flow run"))

        # TODO: Add a Table of the results to your card!
        current.card.append(
            Table(
                rows,  # TODO: What goes here to populate the Table in the card?
                headers=["Model name", "Params", "Task pathspec", "Accuracy", "ROCAUC"],
            )
        )

        fig, ax = plt.subplots(1, 1)
        plt.xticks(rotation=40)
        sns.violinplot(data=violin_plot_df, x="name", y="accuracy", ax=ax)

        # TODO: Append the matplotlib fig to the card
        # Docs: https://docs.metaflow.org/metaflow/visualizing-results/easy-custom-reports-with-card-components#showing-plots
        
        current.card.append(Markdown("### Model accuracy comparison"))
        current.card.append(Image.from_matplotlib(fig))

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == "__main__":
    BaselineChallenge()

Overwriting baseline_challenge.py


In [96]:
! python baseline_challenge.py run --data '../data/Womens Clothing E-Commerce Reviews.csv'

14201.71s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Metaflow 2.9.7.2+ob(v1) executing BaselineChallenge for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Including file ../data/Womens Clothing E-Commerce Reviews.csv of size 8MB 
2023-10-27 21:44:48.475 Workflow starting (run-id 31), see it in the UI at https://ui-pw-1910845203.outerbounds.dev/BaselineChallenge/31
2023-10-27 21:44:48.701 [31/start/141 (pid 6239)] Task is starting.
2023-10-27 21:44:52.711 [31/start/141 (pid 6239)] num of rows in train set: 18112
2023-10-27 21:44:56.119 [31/start/141 (pid 6239)] num of rows in validation set: 4529
2023-10-27 21:44:56.448 [31/start/141 (pid 6239)] Task finished successfully.
2023-10-27 21:44:56.858 [31/baseline/142 (pid 6338)] Task is starting.
2023-10-27 21:44:57.090 [31/model/143 (pid 6341)] Task is starting.
2023-10-27 21:44:59.251 [31/model/143 (pid 6341)] 2023-10-27 21:44:59.251750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAP

## Task 4: Exploring Advanced Visualization Opportunities with MF Cards (Optional)

As ModaMetric continues to thrive and grow, it's clear that basic visualizations won't be enough to understand the intricate dynamics of our e-commerce customer sentiment. We want to take our data storytelling to the next level. And you, as a valued member of our data science team, are the perfect person to lead this initiative.

This optional task is an open invitation for you to really explore how you can leverage Metaflow's features to deliver a compelling, multidimensional story.

### Step 1: Dive Deeper into Hyperparameter Tuning Insights

While we have already visualized the results of the hyperparameter tuning, we believe there's more to unearth. Consider how you might visualize the correlation between specific hyperparameters and model performance, or how different hyperparameter combinations affect the training time.

### Step 2: Unearth Hidden Trends in Customer Sentiment

ModaMetric prides itself on delivering the best for our customers. Can we use our sentiment analysis data to learn more about our customer preferences? Try to create visualizations that show trends in sentiment across different clothing categories, times of year, or any other dimension you find interesting.

### Step 3: Explore Advanced Visualization Techniques

Metaflow can accommodate a wide range of data visualization techniques. This is your chance to showcase those advanced skills. Perhaps you could experiment with multi-panel plots, 3D visualizations, or interactive plots that let viewers explore the data for themselves. You can refer to this [blog post](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjJwOe55pqAAxXA6KACHTZzAsoQFnoECCAQAQ&url=https%3A%2F%2Fouterbounds.com%2Fblog%2Fintegrating-pythonic-visual-reports-into-ml-pipelines%2F&usg=AOvVaw2PY3huULq5xR3yZEQ1s-OL&opi=89978449) for more information about how you may do this. 

We're looking forward to seeing where your creativity and technical expertise can lead ModaMetric. Remember, there are no boundaries - the sky's the limit!

In [97]:
! pip install wordcloud 

14326.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [28]:
%%writefile challenge4.py
from metaflow import (
    FlowSpec,
    step,
    Flow,
    current,
    Parameter,
    IncludeFile,
    card,
    current,
)

from metaflow.cards import Table, Markdown, Artifact, Image
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# move your labeling function from earlier in the notebook here
labeling_function = lambda row: 1 if row['rating'] >= 4 else 0


class NLPFlow(FlowSpec):
    # We can define input parameters to a Flow using Parameters
    # More info can be found here https://docs.metaflow.org/metaflow/basics#how-to-define-parameters-for-flows
    split_size = Parameter("split-sz", default=0.2)
    # In order to use a file as an input parameter for a particular Flow we can use IncludeFile
    # More information can be found here https://docs.metaflow.org/api/flowspec#includefile
    data = IncludeFile("data", default="../data/Womens Clothing E-Commerce Reviews.csv")

    @step
    def start(self):
        # Step-level dependencies are loaded within a Step, instead of loading them
        # from the top of the file. This helps us isolate dependencies in a tight scope.
        import pandas as pd
        import io
        from sklearn.model_selection import train_test_split

        # load dataset packaged with the flow.
        # this technique is convenient when working with small datasets that need to move to remove tasks.
        df = pd.read_csv(io.StringIO(self.data))
        
        # filter down to reviews and labels
        df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
        df["review_text"] = df["review_text"].astype("str")
        _has_review_df = df[df["review_text"] != "nan"]
        self.df_original= _has_review_df
        reviews = _has_review_df["review_text"]
        labels = _has_review_df.apply(labeling_function, axis=1)
        # Storing the Dataframe as an instance variable of the class
        # allows us to share it across all Steps
        # self.df is referred to as a Data Artifact now
        # You can read more about it here https://docs.metaflow.org/metaflow/basics#artifacts
        self.df = pd.DataFrame({"label": labels, **_has_review_df})
        del df
        del _has_review_df

        # split the data 80/20, or by using the flow's split-sz CLI argument
        _df = pd.DataFrame({"review": reviews, "label": labels})
        self.traindf, self.valdf = train_test_split(_df, test_size=self.split_size)
        print(f"num of rows in train set: {self.traindf.shape[0]}")
        print(f"num of rows in validation set: {self.valdf.shape[0]}")

        self.next(self.review_tfidf_vectorizer)


    @step
    def review_tfidf_vectorizer(self):


        from sklearn.feature_extraction.text import TfidfVectorizer

        # Text Vectorization (TF-IDF Vectorizer)
        tfidf_vectorizer = TfidfVectorizer(max_features=1000,stop_words='english') 
        self.X_train = self.traindf['review']
        self.y_train = self.traindf['label']
        self.X_val = self.valdf['review']
        self.y_val = self.valdf['label']
        self.X_train_tfidf = tfidf_vectorizer.fit_transform(self.X_train)
        self.X_val_tfidf = tfidf_vectorizer.transform(self.X_val)
        self.next(self.hyperparameter_tuning)


    @step
    def hyperparameter_tuning(self):
        param_grid = {
                'C': [1.0, 10.0],
                'gamma': ['scale', 'auto']
            }
        grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=3)
        grid_search.fit(self.X_train_tfidf, self.y_train)
        results = grid_search.cv_results_

        # Display the results as a pandas DataFrame (optional)
        self.df_hp = pd.DataFrame(results)
        self.next(self.end)

    @card(
        type="corise"
    )    

    @step
    def end(self):
        print(self.df_hp)
        
        df_result = self.df_hp
        heatmap_data = df_result.pivot(index='param_C', columns='param_gamma', values='mean_test_score')


        # Create a heatmap
        fig = plt.figure(figsize=(10, 6))
        sns.heatmap(heatmap_data, annot=True, cmap='viridis')
        plt.title('Hyperparameter Grid Search Results')
        plt.xlabel('Gamma')
        plt.ylabel('C')
        current.card.append(Image.from_matplotlib(fig))
       
        df_cloud = self.df_original
        text = ' '.join(df_cloud['review_text'])
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
        fig = plt.figure(figsize=(20, 12))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('Word Cloud from Text Column')
        current.card.append(Image.from_matplotlib(fig))

if __name__ == "__main__":
   NLPFlow()


Overwriting challenge4.py


In [29]:
! python challenge4.py run

Metaflow 2.9.7.2+ob(v1) executing NLPFlow for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Including file ../data/Womens Clothing E-Commerce Reviews.csv of size 8MB 
2023-10-27 22:59:02.586 Workflow starting (run-id 36), see it in the UI at https://ui-pw-1910845203.outerbounds.dev/NLPFlow/36
2023-10-27 22:59:02.838 [36/start/166 (pid 10122)] Task is starting.
2023-10-27 22:59:08.153 [36/start/166 (pid 10122)] num of rows in train set: 18112
2023-10-27 22:59:11.836 [36/start/166 (pid 10122)] num of rows in validation set: 4529
2023-10-27 22:59:12.222 [36/start/166 (pid 10122)] Task finished successfully.
2023-10-27 22:59:12.747 [36/review_tfidf_vectorizer/167 (pid 10224)] Task is starting.
2023-10-27 22:59:23.136 [36/review_tfidf_vectorizer/167 (pid 10224)] Task finished successfully.
2023-10-27 22:59:23.473 [36/hyperparameter_tuning/168 (pid 10321)] Task is starting.
2023-10-27 23:04:02.739 1 task is running: hyperparameter_tunin